# Titanic rnd forest

In [1]:
%run prepare_data.py

In [2]:
rnd_forest = RandomForestClassifier(random_state=42, n_jobs=-1)
skfold = StratifiedKFold(n_splits=8)

In [4]:
# value: scores for training and for dev test set, and clf model
clf_cols_scores = {cols : [[],None] for cols in drop_col_lists}

for drop_group in drop_col_lists.items():
    
    pips = []
    for colname in drop_group[0].split(", "):
        pips.append( (colname, cols_pipelines[colname], [colname]) )
    
    attr_pip = ColumnTransformer(pips, remainder="passthrough")
    
    strat_train =  strat_train_all.drop(drop_group[1], axis = 1)
    strat_devtest = strat_devtest_all.drop(drop_group[1], axis = 1)
    
    strat_train_prep = attr_pip.fit_transform(strat_train)
    strat_devtest_prep = attr_pip.fit_transform(strat_devtest)

    n_features = strat_train_prep.shape[1]

    param_grid = [
        {"random_state" : [42], "n_estimators" : [8,100], "bootstrap" : [True, False], 
#          "max_features" : [None] + list(np.arange(n_features)),
         "max_features" : [None] + [5],
         "max_depth" : [None, 3,4,5,6], "max_leaf_nodes" : [None, 3],
         "min_samples_split" : [2,3], "min_samples_leaf" : [1,2,3,4]}
    ]

    gridsearch_nosvd = GridSearchCV(rnd_forest, param_grid = param_grid, cv = skfold, scoring=make_scorer(f1_score),
                              return_train_score=False)
    gridsearch_nosvd.fit(strat_train_prep, strat_train_labels)
    
    
#     pred_scores = cross_val_predict(best_rnd_forest_nosvd, strat_train_prep, strat_train_labels, cv = skfold, method="predict_proba")
#     pred = best_rnd_forest_nosvd.predict(strat_train_prep)
    
#     precision = precision_score(strat_train_labels, pred)
#     recall = recall_score(strat_train_labels, pred)
#     roc_score = roc_auc_score(strat_train_labels, pred_scores[:,1])
#     conf_mat = confusion_matrix(strat_train_labels, pred)
    
#     clf_cols_scores[drop_group[0]][0][0] = [precision, recall, roc_score, conf_mat]
#     clf_cols_scores[drop_group[0]][0][2] = best_rnd_forest_nosvd
    
    
###
#     pred_scores = cross_val_predict(best_rnd_forest_nosvd, strat_devtest_prep, strat_devtest_labels, cv = skfold, method="predict_proba")
#     pred = best_rnd_forest_nosvd.predict(strat_devtest_prep)
    
#     precision = precision_score(strat_devtest_labels, pred)
#     recall = recall_score(strat_devtest_labels, pred)
#     roc_score = roc_auc_score(strat_devtest_labels, pred_scores[:,1])
#     conf_mat = confusion_matrix(strat_devtest_labels, pred)
    
#     clf_cols_scores[drop_group[0]][0][1] = [precision, recall, roc_score, conf_mat]

    
###############
#     svd_rnd_forest_pip = Pipeline([
#         ("svd", TruncatedSVD()),
#         ("rnd_forest", DecisionTreeClassifier())
#     ])
     
#     svd_param_grid = [
#         {"svd__n_components" : [2,3],
#          "rnd_forest__random_state" : [42],  "rnd_forest__n_estimators" : [5,10,100], "rnd_forest__bootstrap" : [True, False], 
#          "rnd_forest__max_features" : [None, 2, 3], "rnd_forest__max_depth" : [None, 3,4,5,6], "rnd_forest__max_leaf_nodes" : [None, 3],
#          "rnd_forest__min_samples_split" : [2,3], "rnd_forest__min_samples_leaf" : [1,2,3,4]}
#     ]
    
#     gridsearch_svd = GridSearchCV(svd_rnd_forest_pip, param_grid = svd_param_grid, cv = skfold,
#                               scoring=make_scorer(f1_score), return_train_score=False)
#     gridsearch_svd.fit(strat_train_prep, strat_train_labels)
     
#     pred_scores = cross_val_predict(best_rnd_forest_svd, strat_train_prep, strat_train_labels, cv = skfold, method="predict_proba")
#     pred = best_rnd_forest_svd.predict(strat_train_prep)
    
#     precision = precision_score(strat_train_labels, pred)
#     recall = recall_score(strat_train_labels, pred)
#     roc_score = roc_auc_score(strat_train_labels, pred_scores[:,1])
#     conf_mat = confusion_matrix(strat_train_labels, pred)
    
#     clf_cols_scores[drop_group[0]][1][0] = [precision, recall, roc_score, conf_mat]
#     clf_cols_scores[drop_group[0]][1][2] = best_rnd_forest_svd
    
    
###
#     pred_scores = cross_val_predict(best_rnd_forest_svd, strat_devtest_prep, strat_devtest_labels, cv = skfold, method="predict_proba")
#     pred = best_rnd_forest_svd.predict(strat_devtest_prep)
    
#     precision = precision_score(strat_devtest_labels, pred)
#     recall = recall_score(strat_devtest_labels, pred)
#     roc_score = roc_auc_score(strat_devtest_labels, pred_scores[:,1])
#     conf_mat = confusion_matrix(strat_devtest_labels, pred)
    
#     clf_cols_scores[drop_group[0]][1][1] = [precision, recall, roc_score, conf_mat]

    if gridsearch_nosvd.best_score_ >= gridsearch_svd.best_score_:
        clf_cols_scores[drop_group[0]][0] = gridsearch_nosvd.best_score_
        clf_cols_scores[drop_group[0]][1] = gridsearch_nosvd.best_estimator_
    else:
        clf_cols_scores[drop_group[0]][0] = gridsearch_svd.best_score_
        clf_cols_scores[drop_group[0]][1] = gridsearch_svd.best_estimator_

KeyboardInterrupt: 

In [ ]:
best_colset_clf = sorted(clf_cols_scores.items(), key = lambda e: e[1][0], reverse=True)[0]
best_rnd_forest = best_colset_clf[1][1]
best_colset_clf

In [ ]:
strat_train = strat_train_all.drop(drop_col_lists[best_colset_clf[0]], axis = 1)
strat_devtest = strat_devtest_all.drop(drop_col_lists[best_colset_clf[0]], axis = 1)

pips = []
for colname in best_colset_clf[0].split(", "):
    pips.append( (colname, cols_pipelines[colname], [colname]) )

attr_pip = ColumnTransformer(pips, remainder="passthrough")

strat_train_prep = attr_pip.fit_transform(strat_train)
strat_devtest_prep = attr_pip.fit_transform(strat_devtest)

## On seen

In [ ]:
pred_scores = cross_val_predict(best_rnd_forest, strat_train_prep, strat_train_labels, cv = skfold, method="predict_proba")
pred = best_rnd_forest.predict(strat_train_prep)

confusion_matrix(strat_train_labels, pred)

In [ ]:
precisions, recalls, thresholds = precision_recall_curve(strat_train_labels, pred_scores[:, 1])

sns.lineplot(thresholds, precisions[:-1], color="r")
sns.lineplot(thresholds, recalls[:-1], color="b")

In [ ]:
plot_precision_recall_curve(best_rnd_forest, strat_train_prep, strat_train_labels, response_method="predict_proba")

In [ ]:
roc_auc_score(strat_train_labels, pred_scores[:,1])

In [ ]:
fpr, tpr, thresh = roc_curve(strat_train_labels, pred_scores[:, 1])
sns.lineplot(x = fpr, y = tpr)

In [ ]:
sns.lineplot(x = thresh, y = fpr)
sns.lineplot(x = thresh, y = tpr, color = "r")

## On unseen

In [ ]:
pred_scores = best_rnd_forest.predict_proba(strat_devtest_prep)
pred = best_rnd_forest.predict(strat_devtest_prep)

confusion_matrix(strat_devtest_labels, pred)

In [ ]:
precisions, recalls, thresholds = precision_recall_curve(strat_devtest_labels, pred_scores[:, 1])

sns.lineplot(thresholds, precisions[:-1], color="r")
sns.lineplot(thresholds, recalls[:-1], color="b")

In [ ]:
plot_precision_recall_curve(best_rnd_forest, strat_devtest_prep, strat_devtest_labels, response_method="predict_proba")

In [ ]:
roc_auc_score(strat_devtest_labels, pred_scores[:,1])

In [ ]:
fpr, tpr, thresh = roc_curve(strat_devtest_labels, pred_scores[:, 1])
sns.lineplot(x = fpr, y = tpr)

In [ ]:
sns.lineplot(x = thresh, y = fpr)
sns.lineplot(x = thresh, y = tpr, color = "r")

## Custom thresholds

In [ ]:
pred_scores = cross_val_predict(best_rnd_forest, strat_train_prep, strat_train_labels, cv = skfold, method="predict_proba")
precisions, recalls, thresholds = precision_recall_curve(strat_train_labels, pred_scores[:, 1])

thresholds[np.argmax(precisions >= 0.9)]

In [ ]:
thresh = thresholds[np.argmax(precisions >= 0.9)]
custom_thresh_preds = pred_scores > thresh
precisions, recalls, thresholds = precision_recall_curve(strat_train_labels, custom_thresh_preds[:, 1])

# sns.lineplot(thresholds, precisions[:-1], color="r")
# sns.lineplot(thresholds, recalls[:-1], color="b")
precision_score(strat_train_labels, custom_thresh_preds[:, 1])

In [ ]:
confusion_matrix(strat_train_labels, custom_thresh_preds[:,1])

# Test set

In [ ]:
#best_model = RandomForestClassifier(random_state=42)
best_model = best_rnd_forest

full_train = titanic.drop("Survived", axis=1)
full_train_prep = attr_pip.fit_transform(full_train)
best_model.fit(full_train_prep, titanic_labels)

In [ ]:
test = pd.read_csv(fname_test)
test_orig = test

class CabinLetterTest(BaseEstimator, TransformerMixin):
    def __init__(self):
        pass
    def fit(self, X, y = None):
        return self
    def transform(self, X):
        df = pd.DataFrame(X, index=test.index, columns=test.columns)
        s_cabin_letters = df.Cabin.str.extract("^([A-Z]).*", expand = False)
        return np.c_[df.to_numpy(), s_cabin_letters]


cabin_letter_pip = Pipeline([
    ("cabin_letter_pip", CabinLetterTest())
])

test = pd.DataFrame(cabin_letter_pip.fit_transform(test), index=test.index,
                       columns = test.columns.union(pd.Index(["Cabin_letter"]), sort=False))
test.loc[ (test.Pclass == 1) & test.Cabin_letter.isna(), "Cabin_letter"] = 'C'
test.loc[ (test.Pclass == 2) & test.Cabin_letter.isna(), "Cabin_letter"] = 'F'
test.loc[ (test.Pclass == 3) & test.Cabin_letter.isna(), "Cabin_letter"] = 'G'

test["Family_size"] = test.SibSp + test.Parch

s_ticket_group = test.groupby("Ticket").size().rename("Ticket_group_size")
test = test.join(s_ticket_group, on="Ticket", sort=False)

test["Fare_p_person"] = pd.Series(np.nan)
test.loc[test.Ticket_group_size == 1, "Fare_p_person"] = test.loc[test.Ticket_group_size == 1, "Fare"]
test.loc[(test.Ticket_group_size > 1), "Fare_p_person"] = \
    test.loc[(test.Ticket_group_size > 1), "Fare"] / (test.loc[(test.Ticket_group_size > 0), "Ticket_group_size"])

test.loc[152, "Fare_p_person"] = 12.0

age_round = test.Age.dropna().apply(np.floor)
test = test.join(age_round, lsuffix="_x", sort=False)
test.drop("Age_x", axis=1, inplace=True)

knnimputer = KNNImputer(n_neighbors=5, missing_values=np.nan)

mask = test.Age.notna() & ( (test.Fare_p_person < 12) | (test.Age < 12) )
df_children_or_low_fare = test.loc[mask, ["Family_size", "Fare_p_person", "Age"]]
knnimputer.fit(df_children_or_low_fare)
missing_age_low_fare = test.loc[(test.Fare_p_person < 12) & test.Age.isna()][["Family_size", "Fare_p_person", "Age"]]
imp = knnimputer.transform(missing_age_low_fare)
test.loc[(test.Fare_p_person < 12) & test.Age.isna(), "Age"] = imp[:, 2]

mask = test.Age.notna() & (test.Fare_p_person >= 12)
df_children_or_low_fare = test.loc[mask, ["Family_size", "Fare_p_person", "Age"]]
knnimputer.fit(df_children_or_low_fare)
missing_age_usual_fare = test.loc[(test.Fare_p_person >= 12) & test.Age.isna()][["Family_size", "Fare_p_person", "Age"]]
imp = knnimputer.transform(missing_age_usual_fare)
test.loc[(test.Fare_p_person >= 12) & test.Age.isna(), "Age"] = imp[:, 2]

test["Fare_cat"] = pd.cut(test.Fare_p_person, bins = [0, 13, 30, 513], include_lowest=True)

test["Age_cat"] = pd.cut(test["Age"].astype(int), bins = [-1, 20, 40, 60, 90])

test.drop(DROP_COLS, inplace=True, axis=1)


In [ ]:
test_prep = attr_pip.fit_transform(test)

survival_pred = best_model.predict(test_prep)


# survival_proba = best_model.predict_proba(test_prep)
# survival_pred = survival_proba > thresh
# survival_pred = survival_pred[:, 1].astype(int)


survival_pred = pd.DataFrame(survival_pred, columns=["Survived"])

id_survival = pd.concat([test_orig.PassengerId, survival_pred], axis=1)
id_survival.set_index("PassengerId", inplace=True)
id_survival.to_csv(fdir_out + "prediction_"  + os.path.basename(__file__) + ".csv")
id_survival